# SVM model for toxic YouTube comments

En este notebook entrenamos un modelo de **SVM (LinearSVC)** para detectar
comentarios de odio/toxicidad en YouTube.

Objetivos:

- Cargar el dataset **preprocesado** (texto limpio/lematizado + etiquetas).
- Definir columna de texto y columna objetivo (`IsToxic` u otra).
- Vectorizar el texto con **TF-IDF** (unigramas y bigramas).
- Entrenar un modelo **LinearSVC** con `class_weight="balanced"`.
- Evaluar el modelo (accuracy, precision, recall, f1, ROC-AUC).
- Guardar el modelo entrenado como **`.pkl`** en `models/`.
- Guardar las métricas en un **`.json`** en `results/` con el formato acordado.


## 1. Imports y configuración

Importamos todas las librerías necesarias para:

- Carga de datos y manejo de rutas (`pandas`, `pathlib`).
- Modelado (`scikit-learn`).
- Guardado de modelo (`joblib`) y métricas (`json`).
- Medición de tiempo y timestamp (`datetime`).


In [1]:
import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
import json
from datetime import datetime

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    roc_auc_score,
    confusion_matrix,
    classification_report,
)

import joblib

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
